# Welding Anomaly Detection with ResNet-50 Binary Classifier

This notebook demonstrates a complete pipeline for welding anomaly detection using:
1. **YOLO** for weld region detection
2. **ResNet-50** fine-tuned as a binary classifier for good/bad weld classification

## Data Split Strategy
- **Training Set**: Used for model training and parameter updates
- **Validation Set**: Used for model selection (early stopping) AND final evaluation
- **No Test Set**: Validation set serves dual purpose for simplicity

## Pipeline Overview
- Extract weld ROIs from images using XML annotations
- Create a balanced dataset with proper train/val splits  
- Fine-tune ResNet-50 with careful data preprocessing and augmentation
- Evaluate using ROC-AUC, PR-AUC, and F1 metrics on validation set

⚠️ **Note**: In production, consider using a separate test set for unbiased final evaluation.

In [ ]:
!pip install -r requirements.txt

In [ ]:
import torch
from pathlib import Path
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")
else:
    print("Using CPU")

In [ ]:
!python cnn_train/yolo/dataset_formatter.py

In [ ]:
!python cnn_train/yolo/yolo_train.py

In [ ]:
# Check if we have the required data structure
base_dir = Path('/teamspace/studios/this_studio')
train_dir = base_dir / 'downloaded_photos' / 'train'
val_dir = base_dir / 'downloaded_photos' / 'val'

if train_dir.exists():
    train_images = list(train_dir.glob('*.jpeg')) + list(train_dir.glob('*.jpg'))
    train_xmls = list(train_dir.glob('*.xml'))
    print(f"Training images: {len(train_images)}")
    print(f"Training XMLs: {len(train_xmls)}")

if val_dir.exists():
    val_images = list(val_dir.glob('*.jpeg')) + list(val_dir.glob('*.jpg'))
    val_xmls = list(val_dir.glob('*.xml'))
    print(f"Validation images: {len(val_images)}")
    print(f"Validation XMLs: {len(val_xmls)}")


# Run the weld extraction script
!python cnn_train/cnn/extract_welds.py

In [ ]:
# Step 3: Train ResNet-50 Binary Classifier
!python cnn_train/cnn/welding_cnn.py --mode "train"

In [ ]:
!python cnn_train/cnn/welding_cnn.py --mode "val"